# Validation of Code Section Designations

There is a built-in database of some standard sections used in various design standards.

In this page, we try to validate the section properties of some designations.
To this end, we mainly use US I and T designations, which are compiled and well documented in the [AISC Shapes Database](https://www.aisc.org/publications/steel-construction-manual-resources/16th-ed-steel-construction-manual/aisc-shapes-database-v16.0/).

## Some Utilities

What we are going to do in the following is to prepare the database.

In [ ]:
import pandas

section_table = pandas.read_excel(
    "https://www.aisc.org/globalassets/product-files-not-searched/manuals/aisc-shapes-database-v16.0.xlsx",
    sheet_name=1,
    storage_options={"User-Agent": "Mozilla/5.0"},
    usecols="A:CF",
)
print(section_table.head())

We define a function to get the area, moment of inertia about the strong and weak axes.

In [2]:
def get_properties(designation: str):
    section = section_table[section_table["AISC_Manual_Label"] == designation]
    return section["A"].values[0], section["Ix"].values[0], section["Iy"].values[0]

## Analysis Template

Next, we define the function to run the analysis and extract the result.

The following is the template model file that will be used.
A few things to note:

1. The material is assumed to be elastic with unit elastic modulus.
2. The default integration scheme is used for each designation.
3. A unit displacement load is applied to axial, strong axis bending and weak axis bending.
4. The `US3DC` category is used, it automatically recentre the section.

In [3]:
template = """
node 1 0 0 0

material Elastic1D 1 1

section US3DC $designation$ 1 1 1

element SingleSection3D 1 1 1

displacement 1 0 1 1 1
displacement 2 0 1 2 1
displacement 3 0 1 3 1

step static 1 1
set ini_step_size 1

analyze

peek node 1

exit
"""

By using the above settings, the resistances on three DoFs are effectively the area and moments of inertia.

## Extract Results

To extract the data, we process the output.

In [4]:
import subprocess
import tempfile


def run_analysis(designation: str):
    with tempfile.NamedTemporaryFile(delete_on_close=False) as fp:
        with open(fp.name, "w") as f:
            f.write(template.replace("$designation$", designation))

        result = (
            subprocess.check_output(["suanpan", "-nu", "-nc", "-f", fp.name])
            .decode("utf-8")
            .splitlines()
        )

    for i, line in enumerate(result):
        if line.startswith("Resistance"):
            return [float(x) for x in result[i + 1].split()]

In [5]:
all_results = {}


def validate(designation: str):
    if result := run_analysis(designation):
        all_results[designation] = [
            x / y for x, y in zip(result, get_properties(designation))
        ]

## Collect All Sections

We can now iterate over all I and T sections.

In [6]:
for _, row in section_table[
    section_table["AISC_Manual_Label"].str.startswith(("W", "M", "S", "HP", "HSS"))
].iterrows():
    designation = row["AISC_Manual_Label"]
    # ignore C sections
    if designation.startswith("MC"):
        continue

    # ignore pipes
    if designation.startswith("HSS") and row["Ht"] == "–":
        continue

    validate(designation)

In [ ]:
import matplotlib.pyplot as plt


fig = plt.figure(figsize=(100, 40), tight_layout=True)


hss_results = {k: v for k, v in all_results.items() if k.startswith("HSS")}
i_results = {k: v for k, v in all_results.items() if not k.startswith("HSS")}

counter = 0


def plot(title, index, results):
    values = [x[index] for x in results.values()]
    min_value = min(values)
    max_value = max(values)
    colors = ["red" if abs(x - 1) > 0.05 else "green" for x in values]
    global counter
    counter += 1
    ax = fig.add_subplot(610 + counter)
    ax.bar(results.keys(), values, color=colors)
    ax.set_ylabel("Numerical/Analytical")
    ax.set_xlabel("Section")
    ax.set_ybound(min_value - 0.02, max_value + 0.01)
    ax.set_xlim(-1, len(results))
    ax.grid()

    for i, v in enumerate(values):
        ax.text(
            i, min_value - 0.01, f"{v:.3f}", horizontalalignment="center", rotation=90
        )

    ax.set_title(title)
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)


plot("Area", 0, hss_results)
plot("Strong Axis Moment of Inertia", 1, hss_results)
plot("Weak Axis Moment of Inertia", 2, hss_results)
plot("Area", 0, i_results)
plot("Strong Axis Moment of Inertia", 1, i_results)
plot("Weak Axis Moment of Inertia", 2, i_results)


## Area

In general, the area can be relatively accurately computed.
However, as all those sections are internally modelled by three flat pieces, the root fillet cannot be accounted for.
As a result, the numerical area is often smaller than the reference value.

Some very light M shapes cannot be well approximated.

## Strong Axis Bending

Some very heavy T sections tend to have poor strong axis moment of inertia.
In this shapes, the thickness of flange accounts for a significant portion of the overall depth.
The normal stress in the flange presents gradient.
In the meantime, there is only one layer of integration points along the thickness of flange, which is not accurate enough in this case.

## Weak Axis Bending

The tapered shapes tend to have more material towards the center, as a result, the weak axis moment of inertia is smaller.
It is modelled by a flat rectangle in numerical models, this overestimate the moment of inertia, mainly for S and ST shapes.

In [8]:
fig.savefig("us.pdf")

The figures can be downloaded: [us](../us.pdf).

The following is the script to generate autocompletion.
Readers can safely ignore.

In [9]:
us_section = {}

for key, value in all_results.items():
    us_section[f"{key}-2D"] = {
        "prefix": key,
        "description": f"US 2D section {key}, accuracy: {value[0]:.2f}/{value[1]:.2f}",
        "body": [
            f"section US2D {key} " + "${1:(1)} ${2:(2)} ${3:[3]} ${4:[4]} ${5:[5]}",
            "# (1) int, unique tag",
            "# (2) int, material tag",
            "# [3] double, scale, default: 1.0",
            "# [4] int, number of integration points, default: 6",
            "# [5] double, eccentricity, default: 0.0",
            "",
        ],
    }
    us_section[f"{key}-3D"] = {
        "prefix": key,
        "description": f"US 3D section {key}, accuracy: {value[0]:.2f}/{value[1]:.2f}/{value[2]:.2f}",
        "body": [
            f"section US3D {key} "
            + "${1:(1)} ${2:(2)} ${3:[3]} ${4:[4]} ${5:[5]} ${6:[6]}",
            "# (1) int, unique tag",
            "# (2) int, material tag",
            "# [3] double, scale, default: 1.0",
            "# [4] int, number of integration points, default: 6",
            "# [5] double, eccentricity of y axis, default: 0.0",
            "# [6] double, eccentricity of z axis, default: 0.0",
            "",
        ],
    }

In [10]:
import json

with open("us_sections_completion.json", "w") as f:
    json.dump({k: v for k, v in sorted(us_section.items())}, f, indent=4)

In [ ]:
highlighting = []
for key in sorted(all_results.keys()):
    highlighting.append(
        {
            "name": "support.constant.section",
            "match": "\\b(?i)" + key.replace(".", "\\.") + "\\b",
        },
    )

with open("us_sections_highlight.json", "w") as f:
    json.dump(highlighting, f, indent=4)

print(
    r"\b(?i)("
    + "|".join([x.replace(".", r"\.") for x in sorted(all_results.keys())])
    + r")\b"
)